In [21]:
# BOILER PLATE, MUST BE RUN ON SUBMIT NODE
%load_ext autoreload
%autoreload 2
import socket
print("Running on host: ", socket.gethostname())

import sys 
lib_path = '/fsx_0/user/tranx/experiments'
if lib_path not in sys.path:
    sys.path.append(lib_path)
#=================================================
    
import plotly.express as px
import plotly.graph_objects as go
import re
import time
import utils 
import json
import glob
import pandas as pd
import os

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Running on host:  submit-1


In [5]:
def run_job(sbatch_base_script, sbatch_overwrite, positional_env_vars):
    """
    sbatch_base = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
    sbatch_overwrite = {
        "job-name": "eval",
    }

    positional_env_vars = [1, 2, 3, 4]
    """

    sbatch_vars_string = []
    for k, v in sbatch_overwrite.items():
        sbatch_vars_string.append(f"--{k}={v}")
    sbatch_vars_string = ' '.join(sbatch_vars_string)

    positional_env_string = " ".join([str(x) for x in positional_env_vars])

    cmd = f"sbatch --parsable {sbatch_vars_string} {sbatch_base_script} {positional_env_string}"
    print(cmd)

    job_id = utils.get_bash_output(cmd, print_output=False)
    job_id = int(job_id)

    return job_id

In [6]:

params = {
    "eval_plan": "Llama31_336px",
    "json_config": "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json",
    "checkpoint_dir": "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch",
    "benchmark_name": "vqa",
    "checkpoint_id": "2000"
}

job = run_job(
    sbatch_base_script="/fsx_0/user/tranx/experiments/eval/sbash_eval.sh",
    sbatch_overwrite={
        "job-name": "eval_" + params["benchmark_name"]
    },
    positional_env_vars=list(params.values())
)

job

sbatch --parsable --job-name=eval_vqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_vqa.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch vqa 2000


9675

# Eval plan

## Llama31

In [42]:
# Llama31
EVAL_PLAN = "Llama31_336px"
BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"
checkpoint_start = 2000
checkpoint_end = 8000
checkpoint_int = 1000

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in range(checkpoint_start, checkpoint_end + checkpoint_int, checkpoint_int):
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(f'job_dict_{EVAL_PLAN}.json', 'w') as f:
    json.dump(job_dict, f, indent=4)

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 2000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 3000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 4000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json 

In [82]:
# Llama31
EVAL_PLAN = "Llama31_336px_8500_9500"
checkpoints = [8500, 9000, 9500]

job_dict_json = f'job_dict_{EVAL_PLAN}.json'
assert not os.path.exists(job_dict_json)

BASE_SCRIPT = "/fsx_0/user/tranx/experiments/eval/sbash_eval.sh"
CONFIG_DIR_LLAMA31 = "/fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31"
CHECKPOINT_DIR_LLAMA31 = "/fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch"

all_benchmarks = "mmmu docvqa mathvista ai2d chartqa vqa textvqa infographics_w_ocr infographics mmbench"

job_dict = {}
for benchmark in all_benchmarks.split():
    job_dict[benchmark] = {}

    for chk in checkpoints:
        params = {
            "eval_plan": EVAL_PLAN,
            "json_config": f"{CONFIG_DIR_LLAMA31}/eval_{benchmark}.json",
            "checkpoint_dir": CHECKPOINT_DIR_LLAMA31,
            "benchmark_name": benchmark,
            "checkpoint_id": str(chk)
        }

        assert os.path.exists(params["json_config"])
        assert os.path.exists(f"{params['checkpoint_dir']}/checkpoint-{chk}")

        job_id = run_job(
            sbatch_base_script=BASE_SCRIPT,
            sbatch_overwrite={
                "job-name": f"eval_{benchmark}"
            },
            positional_env_vars=list(params.values())
        )

        job_dict[benchmark][chk] = int(job_id)

with open(job_dict_json, 'w') as f:
    json.dump(job_dict, f, indent=4)

print(f"Saved job_dict to {job_dict_json}")

sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px_8500_9500 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 8500
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px_8500_9500 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 9000
sbatch --parsable --job-name=eval_mmmu /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px_8500_9500 /fsx_0/user/tranx/eval/llm_mm_aligner/experiments/aws_adel/eval_31/eval_mmmu.json /fsx_0/checkpoints/tranx/MM9-Pretrain-70B/Llama31_336px_128nodes_bz32_scratch mmmu 9500
sbatch --parsable --job-name=eval_docvqa /fsx_0/user/tranx/experiments/eval/sbash_eval.sh Llama31_336px_8500_9500 /fsx_0/user/tranx/eval/llm_mm_aligner/ex

In [73]:
job_dict

{'mmmu': {2000: 9759,
  3000: 9760,
  4000: 9761,
  5000: 9762,
  6000: 9763,
  7000: 9764,
  8000: 9765},
 'docvqa': {2000: 9766,
  3000: 9767,
  4000: 9768,
  5000: 9769,
  6000: 9770,
  7000: 9771,
  8000: 9772},
 'mathvista': {2000: 9773,
  3000: 9774,
  4000: 9775,
  5000: 9776,
  6000: 9777,
  7000: 9778,
  8000: 9779},
 'ai2d': {2000: 9780,
  3000: 9781,
  4000: 9782,
  5000: 9783,
  6000: 9784,
  7000: 9785,
  8000: 9786},
 'chartqa': {2000: 9787,
  3000: 9788,
  4000: 9789,
  5000: 9790,
  6000: 9791,
  7000: 9792,
  8000: 9793},
 'vqa': {2000: 9794,
  3000: 9795,
  4000: 9796,
  5000: 9797,
  6000: 9798,
  7000: 9799,
  8000: 9800},
 'textvqa': {2000: 9801,
  3000: 9802,
  4000: 9803,
  5000: 9804,
  6000: 9805,
  7000: 9806,
  8000: 9807},
 'infographics_w_ocr': {2000: 9808,
  3000: 9809,
  4000: 9810,
  5000: 9811,
  6000: 9812,
  7000: 9813,
  8000: 9814},
 'infographics': {2000: 9815,
  3000: 9816,
  4000: 9817,
  5000: 9818,
  6000: 9819,
  7000: 9820,
  8000: 9821},
 'm

# Parse results

In [109]:
import eval_helper

with open('job_dict_Llama31_336px.json', 'r') as f:
    job_dict = json.load(f)

df = eval_helper.get_eval_scores(job_dict)

Got result for mmmu - 2000
Got result for mmmu - 3000
Got result for mmmu - 4000
Got result for mmmu - 5000
Got result for mmmu - 6000
Got result for mmmu - 7000
Got result for mmmu - 8000
Got result for docvqa - 2000
Got result for docvqa - 3000
Got result for docvqa - 4000
Got result for docvqa - 5000
Got result for docvqa - 6000
Got result for docvqa - 7000
Got result for docvqa - 8000
Got result for mathvista - 2000
Got result for mathvista - 3000
Got result for mathvista - 4000
Got result for mathvista - 5000
Got result for mathvista - 6000
Got result for mathvista - 7000
Got result for mathvista - 8000
Got result for ai2d - 2000
Got result for ai2d - 3000
Got result for ai2d - 4000
Got result for ai2d - 5000
Got result for ai2d - 6000
Got result for ai2d - 7000
Got result for ai2d - 8000
Got result for chartqa - 2000
Got result for chartqa - 3000
Got result for chartqa - 4000
Got result for chartqa - 5000
Got result for chartqa - 6000
Got result for chartqa - 7000
Got result for 